# Decorator Pattern

## Use Case: Ordering Coffe with Add Ons

In [2]:
import inspect
from typing import Callable, Type

def override(interface_class: Type) -> Callable:
    def decorator(method: Callable) -> Callable:
        method_name = method.__name__
        for cls in inspect.getmro(interface_class):
            if cls is not object and method_name in cls.__dict__:
                return method
        raise TypeError(f"{method_name} does not override any method in {interface_class.__name__}")
    return decorator

In [3]:
from abc import ABC, abstractmethod

class Beverage(ABC):

    def __init__(self) -> None:
        self._description = "Unkown Beverage"

    def getDescription(self) -> str:
        return self._description
    
    @abstractmethod
    def getCost(self) -> float:
        raise NotImplementedError
    
class CondimentDecorator(Beverage, ABC):

    def __init__(self, beverage: Beverage) -> None:
        self._beverage: Beverage = beverage

    def getBeverage(self) -> Beverage:
        return self._beverage

    @abstractmethod
    def getDescription(self) -> str:
        raise NotImplementedError

Base Classes

In [4]:
class Expresso(Beverage):

    def __init__(self, cost: float) -> None:
        super().__init__()
        self._description = "Expresso"
        self._cost = cost

    @override(Beverage)
    def getCost(self) -> float:
        return self._cost
    
class HouseBlend(Beverage):

    def __init__(self, cost: float) -> None:
        super().__init__()
        self._description = "House Blend"
        self._cost = cost

    @override(Beverage)
    def getCost(self) -> float:
        return self._cost

Decorator for Base Classes

In [5]:
class Mocha(CondimentDecorator):

    def __init__(self, beverage: Beverage, cost: float) -> None:
        super().__init__(beverage)
        self._description = "Mocha"
        self._cost = cost

    @override(CondimentDecorator)
    def getDescription(self) -> str:
        return self.getBeverage().getDescription() + " " + self._description
    
    @override(Beverage)
    def getCost(self) -> float:
        return self.getBeverage().getCost() + self._cost

class Caramel(CondimentDecorator):

    def __init__(self, beverage: Beverage, cost: float) -> None:
        super().__init__(beverage)
        self._description = "Caramel"
        self._cost = cost

    @override(CondimentDecorator)
    def getDescription(self) -> str:
        return self.getBeverage().getDescription() + " " + self._description
    
    @override(Beverage)
    def getCost(self) -> float:
        return self.getBeverage().getCost() + self._cost

In [6]:
expresso = Expresso(1.0)
expressoMocha = Mocha(expresso, 2.0)
expressoMochaCaramel = Caramel(expressoMocha, 2.0)
print(expressoMochaCaramel.getDescription(), expressoMochaCaramel.getCost())

Expresso Mocha Caramel 5.0


## Use Case: Deprecation Method

In [24]:
class Component(ABC):

    @abstractmethod
    def operation(self) -> None:
        raise NotImplementedError
    
class ComponentDecorator(Component):

    def __init__(self, component: Component) -> None:
        self._component = component

In [28]:
class ConcreteComponent(Component):

    def __init__(self) -> None:
        print("$ Concrete Component Constructor")

    @override(Component)
    def operation(self) -> None:
        return "Concrete Operation"

In [29]:
import warnings

class DeprecatedDecorator(ComponentDecorator):

    def __init__(self, component: Component) -> None:
        super().__init__(component)
        print("$ Deprecated Decorator Constructor")

    @override(Component)
    def operation(self) -> None:
        warnings.warn(
            f"{self._component.__class__.__name__}.operation is deprecated and will be removed in future versions.",
            category=DeprecationWarning,
            stacklevel=2
        )
        return self._component.operation() + " (Deprecated Function)"

In [31]:
concreteComponent = ConcreteComponent()
print(concreteComponent.operation())

concreteComponent = DeprecatedDecorator(concreteComponent)
print(concreteComponent.operation())

$ Concrete Component Constructor
Concrete Operation
$ Deprecated Decorator Constructor
Concrete Operation (Deprecated Function)


C:\Users\MSI GAMING\AppData\Local\Temp\ipykernel_1784\2063089777.py:5: DeprecationWarning: ConcreteComponent.operation is deprecated and will be removed in future versions.
  print(concreteComponent.operation())
